# Redes de Computadores

## Trabalho Prático Tp1
### MpegDashProxy
draft: 0.1

O objetivo deste trabalho é desenvolver um *proxy* HTTP para suporte à reprodução num browser de filmes disponibilizados segundo os princípios do [MPEG-DASH](https://en.wikipedia.org/wiki/Dynamic_Adaptive_Streaming_over_HTTP) (Dynamic Adaptable Streaming over HTTP).

### Contexto
Um servidor HTTP disponibiliza filmes, cada um codificado em diversas qualidades ou resoluções (bitrates). Cada versão de um filme é denominada de pista (*track*) e é composta por segmentos com duração **idêntica** de alguns segundos, mas de dimensão diferente de acordo com a *bitrate* utilizada. A codificação das pistas foi realizada de modo a que o número total de segmentos e a sua duração seja igual em todas as pistas de um mesmo filme. Adicionalmente, temporalmente, os segmentos estão perfeitamente alinhados entre pistas, pelo que são funcionalmente equivalentes.

A reprodução de um filme realiza-se no *browser* por recurso a um reprodutor (*player*) *javascript*. Para que a reprodução de um filme seja correta, é necessário que os segmentos de um filme sejam entregues ao *player* **atempadamente**, na ordem correta, mas independentemente da pista a que pertencem. 

O diálogo entre o *browser* (*player* *javascript*) e o servidor HTTP onde estão armazenados os filmes será mediado por um *proxy* HTTP. A função deste *proxy* será adequar a qualidade dos segmentos entregues ao *player*, dinamicamente, em função do débito da rede disponível, mas evitando que a reprodução sofra interrupções por falta de dados. Monitorizando o estado da rede, em momentos favoráveis, espera-se que o *proxy* dê prioridade aos segmentos
de pistas de qualidade mais elevada, fazendo o oposto para reagir à degradação do débito da rede ou havendo o risco de o *player* estar prestes a parar, transferindo do servidor segmentos de pior qualidade, mas de menor dimensão.

### Materiais fornecidos

Para a realização deste trabalho são fornecidos:

+ Um servidor HTTP para servir os filmes;
+ Dois pequenos filmes codificados em várias qualidades, nos formatos adequados;
+ Uma página *web* que implementa o reprodutor de filmes e que será o cliente do *proxy* HTTP a desenvolver;
+ [Codigo Java 17]() com um esboço do *proxy* a completar pelos alunos;
+ Quatro imagens *Docker* para executar o servidor HTTP, simulando cenários distintos de rede com débito variável.
